In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, minmax_scale
from keras import layers
from keras.layers import Embedding, Dense, Merge, Dropout, Reshape, Flatten
from keras.models import Sequential
from keras.preprocessing.text import one_hot

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df_user = pd.read_csv('users.csv')
df_business = pd.read_csv('business.csv')
df_review = pd.read_csv('train_reviews.csv')
# df_review.head()

In [3]:
df_user = df_user[['user_id', 'average_stars', 'fans', 'funny', 'name', 'review_count', 'useful']]
df_business = df_business[['business_id', 'address', 'attributes_Alcohol', 'attributes_BikeParking', 'attributes_BusinessAcceptsCreditCards', 'attributes_Caters', 'attributes_GoodForKids', 'attributes_HasTV', 'attributes_NoiseLevel', 'attributes_OutdoorSeating', 'latitude', 'longitude', 'review_count', 'stars']]
df_review = df_review[['user_id', 'business_id', 'stars']]

In [4]:
le = LabelEncoder()
df_user[['name']] = df_user[['name']].astype(str)
df_user['name'] = le.fit_transform(df_user['name'].values)
df_business[['address', 'attributes_Alcohol', 'attributes_BikeParking', 'attributes_BusinessAcceptsCreditCards', \
             'attributes_Caters', 'attributes_GoodForKids', 'attributes_HasTV', 'attributes_NoiseLevel', \
             'attributes_OutdoorSeating']] \
= df_business[['address', 'attributes_Alcohol', 'attributes_BikeParking', 'attributes_BusinessAcceptsCreditCards', \
             'attributes_Caters', 'attributes_GoodForKids', 'attributes_HasTV', 'attributes_NoiseLevel', \
             'attributes_OutdoorSeating']].astype(str)
df_business[['address', 'attributes_Alcohol', 'attributes_BikeParking', 'attributes_BusinessAcceptsCreditCards', \
             'attributes_Caters', 'attributes_GoodForKids', 'attributes_HasTV', 'attributes_NoiseLevel', \
             'attributes_OutdoorSeating']]\
= df_business[['address', 'attributes_Alcohol', 'attributes_BikeParking', 'attributes_BusinessAcceptsCreditCards', \
             'attributes_Caters', 'attributes_GoodForKids', 'attributes_HasTV', 'attributes_NoiseLevel', \
             'attributes_OutdoorSeating']].apply(le.fit_transform)



In [5]:
# df_user.set_index('user_id', inplace=True)
# df_business.set_index('business_id', inplace=True)
# minmax_scale(df_user.values)
# minmax_scale(df_business.values)

In [6]:
# df_user.reset_index(inplace=True)
# df_business.reset_index(inplace=True)
df_user_train = pd.merge(df_user, df_review, how='inner', on=['user_id'])[['user_id', 'average_stars', 'fans', 'funny', 'name', 'review_count', 'useful']]
df_business_train = pd.merge(df_business, df_review.drop(columns=['user_id', 'stars']), how='inner', on=['business_id'])

In [7]:
# df_user_train.set_index('user_id', inplace=True)
# df_business_train.set_index('business_id', inplace=True)
# df_user_train.head()
# df_business_train.head()

In [8]:
# train_user = minmax_scale(df_user_train.values)
# train_business =minmax_scale(df_business_train.values)

In [9]:
df_user_train['user_id'] = df_user_train['user_id'].astype('category').cat.codes.values
df_business_train['business_id'] = df_business_train['business_id'].astype('category').cat.codes.values

In [13]:
num_user = len(df_user_train['user_id'].unique())
num_business = len(df_business_train['business_id'].unique())

In [29]:
def my_model():
    k = 128
    model1 = Sequential()
    model1.add(Embedding(num_user + 1, k, input_length = 1))
    model1.add(Flatten())
    model2 = Sequential()
    model2.add(Embedding(num_business + 1, k, input_length = 1))
    model2.add(Flatten())

    model = Sequential()
    model.add(Merge([model1, model2], mode = 'concat'))
    model.add(Dropout(0.2))
    model.add(Dense(k, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(int(k/4), activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(int(k/16), activation = 'relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(1, activation = 'relu'))
    return model

In [30]:
model = my_model()
model.compile(loss = 'mse', optimizer = "adam")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  # This is added back by InteractiveShellApp.init_path()


In [16]:
X_train = [df_user_train['user_id'], df_business_train['business_id']]
y_train = df_review['stars'].values

In [31]:
model.fit(X_train, y_train, batch_size = 100, epochs = 10)

Epoch 1/10
150232/150232 [==============================] - 78s 521us/step - loss: 3.9454
Epoch 2/10
150232/150232 [==============================] - 78s 523us/step - loss: 2.2719
Epoch 3/10
150232/150232 [==============================] - 87s 578us/step - loss: 1.6647
Epoch 4/10
150232/150232 [==============================] - 91s 603us/step - loss: 1.3875
Epoch 5/10
150232/150232 [==============================] - 88s 587us/step - loss: 1.3004
Epoch 6/10
150232/150232 [==============================] - 91s 607us/step - loss: 1.2771
Epoch 7/10
150232/150232 [==============================] - 87s 577us/step - loss: 1.2608
Epoch 8/10
150232/150232 [==============================] - 89s 594us/step - loss: 1.2473
Epoch 9/10
150232/150232 [==============================] - 96s 641us/step - loss: 1.2354
Epoch 10/10
150232/150232 [==============================] - 86s 575us/step - loss: 1.2275


In [ ]:
# df_user.set_index('user_id', inplace = True)
# df_business.set_index('business_id', inplace=True)

In [18]:
queries = pd.read_csv('test_queries.csv')
queries['user_id'] = queries['user_id'].astype('category').cat.codes.values
queries['business_id'] = queries['business_id'].astype('category').cat.codes.values
queries.head()

,user_id,business_id
0,7469,6948
1,3324,2632
2,2141,5654
3,9212,6396
4,4647,7806


In [19]:
# df_user_test = df_user.loc[queries['user_id']]
# df_business_test = df_business.loc[queries['business_id']]
# df_user_test.head()
# df_business_test.head()

,business_id,address,attributes_Alcohol,attributes_BikeParking,attributes_BusinessAcceptsCreditCards,attributes_Caters,attributes_GoodForKids,attributes_HasTV,attributes_NoiseLevel,attributes_OutdoorSeating,latitude,longitude,review_count,stars
6948,OaM2Bjeo2Ftt84ruTrzPNQ,6949,3,0,1,1,1,0,3,1,33.347893,-111.976822,116,4.5
2632,4aF5GfnVkJPj9IwJ2yxPtQ,8885,1,1,1,0,0,1,0,1,33.423346,-111.941376,335,3.5
5654,vYKZHWBuBT1PUevggLAE9w,10123,3,1,1,1,1,1,3,1,33.421881,-111.925474,111,3.5
6396,O0UhNI9KHdY1wOdhEtT2vg,10523,3,1,1,1,1,1,0,1,33.711445,-112.273148,102,4.0
7806,kosTPb88O4Q0XGbVbEOGCA,8000,2,1,1,2,2,2,2,2,36.086364,-115.035941,113,4.0


In [23]:
X_test = [queries['user_id'], queries['business_id']]

In [33]:
res = np.rint(model.predict(X_test))
res[:100]

array([[3.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [3.],
       [3.],
       [3.],
       [3.],
       [4.],
       [3.],
       [4.],
       [3.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [3.],
       [5.],
       [3.],
       [4.],
       [3.],
       [4.],
       [4.],
       [3.],
       [4.],
       [3.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [3.],
       [4.],
       [3.],
       [4.],
       [3.],
       [4.],
       [4.],
       [3.],
       [3.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [3.],
       [3.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],
       [4.],